In [6]:
import os
import warnings
warnings.filterwarnings('ignore')

os.environ['SERPER_API_KEY'] = ""
os.environ['OPENAI_API_KEY'] = ""

# Serper
- 구글 웹검색 결과를 제공하는 API
- https://serper.dev/?utm_term=serpapi&gad_source=1&gad_campaignid=18303173259&gbraid=0AAAAAo4ZGoGN8cK_1I7Fr3405Mwc8z46r&gclid=Cj0KCQjwotDBBhCQARIsAG5pinMaqaxC68P-wQY2CYZ4vanUy3NjIjhLx1mC5dRyPWvPy_06eMdxZegaAkm2EALw_wcB

In [7]:
from crewai import Agent, Task, Crew
from crewai.process import Process
from crewai_tools import (
    SerperDevTool, # 서퍼 API를 활용한 결과물을 받아볼 수 있는 툴, 특정 키워드를 줬을 떄 해당 키워드를 구글에 검색한 결과/웹검색 결과를 가져오는 도구
    WebsiteSearchTool, # 특정 키워드 그리고 어떤 웹사이트를 주면 해당 웹사이트 내에서 해당 키워드와 관련이 있는 부분만 가져오는, RAG가 내부적으로 실행이 되는 툴 / 
    # 웹사이트 전체를 긁어오는게 아니라 내가 정해준 키워드와 관련된 부분만 가져올 수 있도록 하는 툴
    ScrapeWebsiteTool) # 어떤 URL에 있는 모든 텍스트를 스크리핑 해서 가져오는 도구 / 크롤링을 수행한 결과를 받아볼 수 있음

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="테크 트렌드 연구원",
    goal="인공지능 분야의 최신 기술 트렌드를 한국어로 제공합니다. 지금은 2025년 5월입니다.",
    backstory="기술 트렌드에 예리한 안목을 지닌 전문 분석가이자 AI 개발자입니다.", # role과 goal에서 담아주지 못한 내용을 담아주면 좋음!
    tools=[search_tool, web_rag_tool],
    verbose=True,
    max_iter=5, # 해당 에이전트가 주어진 태스크를 수행하는데 있어서 반복 실행활 횟수를 제한하는 것
    llm=llm
)

writer = Agent(
    role="뉴스레터 작성자",
    goal="최신 AI기술 트렌드에 대한 매력적인 테크 뉴스레터를 한국어로 작성하세요. 지금은 2025년 5월입니다.",
    backstory="기술 트렌드에 예리한 안목을 지닌 전문 분석가이자 AI 개발자입니다.",
    tools=[search_tool, web_rag_tool],
    verbose=True,
    allow_delegation=False, # 해당 에이전트가 주어진 테스크를 수행하는데 있어서 나 혼자만으로 힘들다 라고 한다면 다른 에이전트에게 해당 테스크를 위임하는 파라미터
    llm=llm
)

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\alembic\config.py:564: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


In [9]:
# Task 정의
research = Task(
    description="AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.",
    expected_output="AI 업계에세 가장 주목받는 3개 기술 개발 동향과 그 중요성에 대한 신선한 관점을 요약한 글",
    agent=researcher
)

write = Task(
   description="""테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.
   테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.
   """,
    expected_output="최신 기술 관련 소식을 재밌는 말투로 소개하는 4문단짜리 마크다운 형식 뉴스레터",
    agent=writer,
    output_file="./20250526_AI TECH NEWSLETTER.md"
)

In [ ]:
# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    process=Process.sequential
)

# Execute tasks
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 083dfa41-7490-427d-8b85-554738ded82b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: 테크 트렌드 연구원

## Task: AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.

Output()

In [ ]:
from IPython.display import Markdown, display
display(Markdown(result.raw))